In [5]:
import sys
import torch

import os
import matplotlib.pyplot as plt
import numpy as np
from mobile_sam import sam_model_registry, SamPredictor

sam_checkpoint = "C:/Users/gbo10/OneDrive/measurement_paper_images/sam_experiments/mobile_sam.pt"
model_type = "vit_t"

device = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
sam.eval()

predictor = SamPredictor(sam)

c:\Users\gbo10\Videos\research\counting_research_algorithms\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\gbo10\Videos\research\counting_research_algorithms\.venv\lib\site-packages\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\Users\gbo10\Videos\research\counting_research_algorithms\.venv\lib\site-packages\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if thi

In [6]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))   

In [3]:
from ultralytics import RTDETR

# Load a pretrained YOLOv8n model
model = RTDETR("C:/Users/gbo10/OneDrive/measurement_paper_images/sam_experiments/right_round_best.pt")

# Run inference on 'bus.jpg' with arguments
model.predict("C:/Users/gbo10/OneDrive/measurement_paper_images/to colab/test with seperation/images/right round/GX010068_26_666-jpg_gamma_jpg.rf.d9631e54ca955ee3b93f9ad9bf8b3c5d.jpg", save=True,save_txt=True, imgsz=640, conf=0.25)


image 1/1 C:\Users\gbo10\OneDrive\measurement_paper_images\to colab\test with seperation\images\right round\GX010068_26_666-jpg_gamma_jpg.rf.d9631e54ca955ee3b93f9ad9bf8b3c5d.jpg: 640x640 5 prawns, 2301.3ms
Speed: 8.2ms preprocess, 2301.3ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\detect\predict
1 label saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\detect\predict\labels


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'prawn'}
 obb: None
 orig_img: array([[[130,  91, 153],
         [123,  86, 132],
         [118,  84,  95],
         ...,
         [146, 137,  99],
         [140, 132,  95],
         [136, 128,  91]],
 
        [[129,  90, 152],
         [123,  87, 131],
         [117,  83,  94],
         ...,
         [145, 136,  98],
         [139, 131,  94],
         [133, 125,  88]],
 
        [[130,  89, 150],
         [124,  85, 130],
         [118,  82,  94],
         ...,
         [144, 138,  97],
         [138, 131,  92],
         [131, 124,  85]],
 
        ...,
 
        [[ 54,  29,  49],
         [ 50,  28,  47],
         [ 48,  25,  47],
         ...,
         [ 85,  75,  57],
         [ 84,  74,  57],
         [ 84,  74,  56]],
 
        [[ 54,  26,  49],
         [ 52,  26,  50],
         [ 52,  26,  50],
         ...,
         [ 82,  75,

In [8]:
import numpy as np

def yolo_to_box(yolo_line, img_width, img_height):
    # YOLO format: class x_center y_center width height (normalized)
    yolo_data = yolo_line.strip().split()
    x_center, y_center, width, height = map(float, yolo_data[1:])

    # Convert to pixel coordinates
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height

    # Calculate top-left and bottom-right coordinates
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return np.array([x_min, y_min, x_max, y_max])

def convert_yolo_to_boxes(yolo_file, img_width, img_height):
    boxes = []
    with open(yolo_file, 'r') as file:
        for line in file:
            box = yolo_to_box(line, img_width, img_height)
            boxes.append(box)
    return boxes

# Example usage
yolo_file = 'C:/Users/gbo10/Videos/data-science/Research-counting-algorithms/runs/detect/predict/labels/GX010068_26_666-jpg_gamma_jpg.rf.d9631e54ca955ee3b93f9ad9bf8b3c5d.txt'
img_width = 640  # Image width
img_height = 640  # Image height

boxes = convert_yolo_to_boxes(yolo_file, img_width, img_height)
for box in boxes:
    print(f"input_box = np.array({box})")




input_box = np.array([146 433 238 587])
input_box = np.array([311 343 377 520])
input_box = np.array([363 507 464 603])
input_box = np.array([ 60 348 104 631])
input_box = np.array([ 47 549 169 640])


In [10]:
import cv2

image = cv2.imread('C:/Users/gbo10/OneDrive/measurement_paper_images/to colab/test with seperation/images/right round/GX010068_26_666-jpg_gamma_jpg.rf.d9631e54ca955ee3b93f9ad9bf8b3c5d.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


predictor.set_image(image)

In [11]:
for input_box in boxes:
    masks, _, _ = predictor.predict(
        point_coords=None,
        point_labels=None,
        box=input_box[None, :  ],
        multimask_output=True,
    )
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    show_mask(masks[0], plt.gca())
    show_box(input_box, plt.gca())
    plt.axis('off')
    plt.show()

TypeError: predict() got an unexpected keyword argument 'boxes'